In [1]:
# save as etf_correlation.py and run: python etf_correlation.py
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
from yfinance.exceptions import YFRateLimitError
import matplotlib.pyplot as plt
import os

In [2]:
# Use Tickers (Xetra/EUR), not ISINs
tickers = [
    "INRG.L", # Global Clean Energy
    "URTH", # MSCI World
    "IEUR", # MSCI Europe
    "EEM", # MSCI Emerging Markets
    "IVE", # S&P 500
]

In [3]:
# give small example to get some data from yfinance to test the connection
# retry/backoff settings to avoid Yahoo rate limits
MAX_RETRIES = 3
PAUSE_BETWEEN_CALLS = 10.0
PAUSE_ON_RATE_LIMIT = 60.0
def fetch_one(ticker):
    retries = 0
    while True:
        try:
            etf = yf.Ticker(ticker)
            hist = etf.history(period="1mo", interval="1d")
            return hist
        except YFRateLimitError:
            if retries >= MAX_RETRIES:
                raise
            retries += 1
            print(f"Rate limit hit. Retrying in {PAUSE_ON_RATE_LIMIT} seconds...")
            time.sleep(PAUSE_ON_RATE_LIMIT)
        except Exception as e:
            if retries >= MAX_RETRIES:
                raise
            retries += 1
            print(f"Error fetching data for {ticker}: {e}. Retrying in {PAUSE_BETWEEN_CALLS} seconds...")
            time.sleep(PAUSE_BETWEEN_CALLS)

for ticker in tickers:
    print(f"Fetching data for {ticker}...")
    hist = fetch_one(ticker)
    print(hist.head())


Fetching data for INRG.L...
                                 Open        High         Low       Close  \
Date                                                                        
2025-11-10 00:00:00+00:00  783.433296  792.182551  778.183743  779.933594   
2025-11-11 00:00:00+00:00  788.682834  789.932728  770.934346  776.183899   
2025-11-12 00:00:00+00:00  776.683818  785.683051  767.434606  771.434265   
2025-11-13 00:00:00+00:00  771.000000  773.000000  733.250000  734.250000   
2025-11-14 00:00:00+00:00  731.250000  748.000000  715.000000  746.250000   

                           Volume  Dividends  Stock Splits  Capital Gains  
Date                                                                       
2025-11-10 00:00:00+00:00  341520     0.0000           0.0            0.0  
2025-11-11 00:00:00+00:00  732041     0.0000           0.0            0.0  
2025-11-12 00:00:00+00:00  219124     0.0000           0.0            0.0  
2025-11-13 00:00:00+00:00  203038     6.5694        

In [4]:
end_date = datetime.now()
start_date = end_date - timedelta(days=5 * 365) # Approximately 5 years

print(f"Fetching data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

Fetching data from 2020-12-10 to 2025-12-09


In [5]:
# Modified fetch_one function to accept start and end dates
def fetch_one_historical(ticker, start, end):
    retries = 0
    while True:
        try:
            etf = yf.Ticker(ticker)
            hist = etf.history(start=start, end=end, interval="1d")
            return hist
        except YFRateLimitError:
            if retries >= MAX_RETRIES:
                raise
            retries += 1
            print(f"Rate limit hit for {ticker}. Retrying in {PAUSE_ON_RATE_LIMIT} seconds...")
            time.sleep(PAUSE_ON_RATE_LIMIT)
        except Exception as e:
            if retries >= MAX_RETRIES:
                raise
            retries += 1
            print(f"Error fetching data for {ticker}: {e}. Retrying in {PAUSE_BETWEEN_CALLS} seconds...")
            time.sleep(PAUSE_BETWEEN_CALLS)

all_etf_data = {}
for ticker_symbol in tickers:
    print(f"Fetching historical data for {ticker_symbol}...")
    data = fetch_one_historical(ticker_symbol, start_date, end_date)
    if not data.empty:
        all_etf_data[ticker_symbol] = data
    else:
        print(f"No data found for {ticker_symbol} in the specified period.")
    time.sleep(PAUSE_BETWEEN_CALLS) # Pause between calls to avoid rate limiting

print("Finished fetching all historical data.")

Fetching historical data for INRG.L...
Fetching historical data for URTH...
Fetching historical data for IEUR...
Fetching historical data for EEM...
Fetching historical data for IVE...
Finished fetching all historical data.


In [6]:
# Combine all ETF data into a single DataFrame
combined_df = pd.DataFrame()
for ticker_symbol, df in all_etf_data.items():
    df['Ticker'] = ticker_symbol
    combined_df = pd.concat([combined_df, df])

# Reset index to make 'Date' a regular column
combined_df.reset_index(inplace=True)

# Display the first few rows of the combined DataFrame
display(combined_df.head())

# Export to CSV
output_filename = "etf_historical_data.csv"
combined_df.to_csv(output_filename, index=False)
print(f"Historical data exported to {output_filename}")

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Ticker
0,2020-12-10 00:00:00+00:00,1008.553612,1012.052063,992.560690,1008.053833,741362,0.0,0.0,0.0,INRG.L
1,2020-12-11 00:00:00+00:00,1012.551743,1030.543777,1003.555725,1023.546875,510645,0.0,0.0,0.0,INRG.L
2,2020-12-14 00:00:00+00:00,1026.545502,1035.541519,1019.548600,1032.542847,918526,0.0,0.0,0.0,INRG.L
3,2020-12-15 00:00:00+00:00,1036.041426,1064.528817,1025.546071,1059.031250,839632,0.0,0.0,0.0,INRG.L
4,2020-12-16 00:00:00+00:00,1065.028788,1074.024807,1045.037633,1057.532104,1235100,0.0,0.0,0.0,INRG.L


Historical data exported to etf_historical_data.csv
